<a href="https://colab.research.google.com/github/0xpix/CERI-Project/blob/main/notebooks/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>